# Deepfake Detection - Google Colab Setup

## ⚠️ IMPORTANT: Setup Instructions

This notebook supports multiple setup methods to handle different scenarios.

### **Choose Your Setup Method:**

1. **If repository is PUBLIC on GitHub:**
   - Just run the cells in order
   - The notebook will auto-clone the repo

2. **If repository is PRIVATE or Network Issues:**
   - Upload the entire `Final-year-project` folder to **Google Drive/MyDrive/**
   - Run the cells - notebook will detect and use it
   - See cell below for details

3. **Just Upload Project Files:**
   - Create a ZIP of your project
   - Upload to Colab directly or to Drive
   - Extract and run setup

---

## Setup Status Check

In [ ]:
# Check environment and connection status
print("=" * 60)
print("COLAB ENVIRONMENT CHECK")
print("=" * 60)

import sys
import os

# Check Colab
try:
    from google.colab import drive
    print("✅ Running in Google Colab")
    IN_COLAB = True
except ImportError:
    print("❌ Not running in Google Colab")
    IN_COLAB = False

# Check internet connection
try:
    import subprocess
    result = subprocess.run(['ping', '-c', '1', 'github.com'], 
                          capture_output=True, timeout=5)
    if result.returncode == 0:
        print("✅ Internet connection available")
    else:
        print("⚠️ Cannot reach GitHub (network issue)")
except:
    print("⚠️ Network check failed")

# Check GPU
try:
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️ GPU not available (using CPU)")
except:
    print("⚠️ PyTorch not installed yet")

print("\n📋 Next steps:")
print("1. If GitHub clone fails → Upload project to Google Drive/MyDrive/")
print("2. Extract uploaded ZIP if using that method")
print("3. Run the 'Setup Environment' cell below")
print("=" * 60)

## 1. Setup Environment

In [ ]:
# Setup: Choose method based on your situation

# METHOD 1: Clone from GitHub (requires public repo)
try:
    !git clone https://github.com/Snaju003/Final-year-project.git
    %cd Final-year-project
    print("✅ Repository cloned successfully")
except Exception as e:
    print(f"⚠️ GitHub clone failed: {e}")
    print("Using METHOD 2: Upload from Google Drive instead")

# METHOD 2: If GitHub doesn't work, mount Google Drive and use files from there
import os
if not os.path.exists('Final-year-project'):
    print("\n📁 Mounting Google Drive for file access...")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    
    # Copy project from Google Drive if it exists
    import shutil
    if os.path.exists('/content/drive/MyDrive/Final-year-project'):
        shutil.copytree('/content/drive/MyDrive/Final-year-project', '/content/Final-year-project')
        %cd /content/Final-year-project
        print("✅ Project copied from Google Drive")
    else:
        print("⚠️ Project not found in Google Drive")
        print("📋 To fix: Upload the 'Final-year-project' folder to Google Drive/MyDrive/")

# Verify we're in the right directory
import os
print(f"\nCurrent directory: {os.getcwd()}")
print(f"Files: {os.listdir('.')[:10]}")

In [ ]:
# Install dependencies from requirements.txt
import os
import subprocess
import sys

# Try to install from requirements.txt
if os.path.exists('requirements.txt'):
    print("📦 Installing from requirements.txt...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], 
                   capture_output=True)
    print("✅ Dependencies installed")
else:
    # Fallback: install key packages individually
    print("⚠️ requirements.txt not found. Installing essential packages...")
    packages = [
        'torch==2.0.1',
        'torchvision==0.15.2',
        'timm==0.9.12',
        'facenet-pytorch==2.5.3',
        'opencv-python==4.8.1.78',
        'tqdm==4.65.0',
        'pytorch-gradcam==0.2.1',
        'scikit-image==0.21.0',
        'scikit-learn==1.3.2',
        'Pillow==10.0.1',
    ]
    for pkg in packages:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', pkg],
                       capture_output=True)
    print("✅ Essential packages installed")

In [ ]:
# Check GPU availability
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Using CPU")

## 2. Import Libraries

In [ ]:
import sys
import os
sys.path.insert(0, '/content/Final-year-project')

import torch
import torch.nn as nn
from pathlib import Path
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# Import project modules
from src.models import build_all_models
from src.inference import EnsembleModel

## 3. Load Model and Run Inference

In [ ]:
# Load the ensemble model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnsembleModel().to(device)

# Load pre-trained weights if available
model_path = 'models/ensemble/ensemble_final.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ Model loaded from {model_path}")
else:
    print(f"⚠️ Model weights not found. Using random initialization.")

model.eval()
print(f"✅ Model ready on {device}")

In [ ]:
# Function to predict on a single image
def predict_image(image_path, model, device):
    """
    Predict deepfake probability for a single image
    
    Args:
        image_path: Path to image file
        model: Ensemble model
        device: torch device
    
    Returns:
        deepfake_probability: Float between 0 and 1
    """
    from torchvision import transforms
    
    # Image preprocessing
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    # Inference
    with torch.no_grad():
        output = model(image)
    
    return output.item()

# Example usage (uncomment when you have an image)
# image_path = 'your_image.jpg'
# prob = predict_image(image_path, model, device)
# print(f"Deepfake probability: {prob:.4f}")

## 4. Batch Processing

In [ ]:
def batch_predict(image_dir, model, device):
    """
    Predict on multiple images in a directory
    """
    from torchvision import transforms
    from pathlib import Path
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    results = {}
    image_paths = list(Path(image_dir).glob('*.jpg')) + list(Path(image_dir).glob('*.png'))
    
    for image_path in tqdm(image_paths, desc="Processing images"):
        try:
            image = Image.open(image_path).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = model(image)
            
            results[image_path.name] = output.item()
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    
    return results

# Example usage
# results = batch_predict('path/to/images', model, device)
# for img_name, prob in results.items():
#     print(f"{img_name}: {prob:.4f}")

## 5. Download Results

In [ ]:
# Download results to local machine
from google.colab import files

# Save results to a file
import json
# results_path = 'results.json'
# with open(results_path, 'w') as f:
#     json.dump(results, f, indent=2)

# Download
# files.download(results_path)